<a href="https://colab.research.google.com/github/caua-sathler/NEURAL-NETWORKS/blob/main/FL-DP-PSO-SGD/FLPSO_SGD_DP_Clients_gradient_clipping_balanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from torchvision import datasets, transforms
import torch.nn.functional as F
import copy
import random
import torchvision
import torchvision.models as models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
import numpy as np

In [2]:
class MLP(nn.Module):

    def __init__(self, device, input_size=28*28, hidden_size=256, num_classes=10):
        super(MLP, self).__init__()
        self.device = device
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.to(device)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Achatar o tensor de entrada
        y = self.fc1(x)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.relu(y)
        y = self.fc3(y)

        return y

In [3]:

# Definições dos hiperparâmetros
NUM_CLIENTES = 5
NUM_PARTICULAS = 20
NUM_RODADAS = 10
INERCIA, C1, C2 = 0.9, 0.8, 0.9
EPSILON = 6/math.sqrt(50)
DELTA = 1e-5
SENSITIVITY = 4
MAX_NORM = 2.0
# SUBSET_SIZE = 12000
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'training on {DEVICE}')

# Criando o modelo global
modelo_global = MLP(DEVICE)
criterio = nn.CrossEntropyLoss()

training on cuda


In [8]:
#Seeds para reprodutibilidade
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

class Particula:
    def __init__(self, particle_id, modelo_cliente):
        self.particle_id = particle_id
        self.pesos = copy.deepcopy(modelo_cliente.state_dict())  # Conjunto de pesos da partícula
        self.melhor_pesos = copy.deepcopy(self.pesos)  # pbest (melhor posição da partícula)
        self.melhor_erro = float('inf')  # Melhor erro alcançado
        self.velocidade = {name: torch.zeros_like(param) for name, param in self.pesos.items()}  # Velocidade do PSO
        self.device = modelo_cliente.device  # Dispositivo do modelo

    def atualizar_pso(self, global_best_pesos, INERCIA, C1, C2):
        """Atualiza os pesos da partícula usando a equação do PSO."""
        for name in self.pesos:
            local_rand = random.random()
            global_rand = random.random()
            self.velocidade[name] = (
                INERCIA * self.velocidade[name] +
                C1 * local_rand * (self.melhor_pesos[name] - self.pesos[name]) +
                C2 * global_rand * (global_best_pesos[name] - self.pesos[name])
            )
            self.pesos[name] += self.velocidade[name]

            # ''' sigma = SENSITIVITY * torch.sqrt((2.0 * torch.log(torch.tensor(1.0 / DELTA))).clone().detach()) / EPSILON

            #   # Gerar ruído diretamente com a distribuição normal do PyTorch (muito mais eficiente!)
            #   noise = torch.normal(mean=0, std=sigma, size=self.velocidade[name].shape, device=self.device)
            #   self.velocidade[name] += noise
            #   #clipping velocity
            #   self.velocidade[name] = torch.clamp(self.velocidade[name], -MAX_VELOCITY, MAX_VELOCITY)'''

    def avaliar_perda(self, modelo_cliente, criterio, dados):
        """Calcula a perda da partícula no modelo do cliente."""
        modelo_cliente.load_state_dict(self.pesos)  # Aplica os pesos da partícula no modelo do cliente
        modelo_cliente.eval()
        total_loss = 0

        with torch.no_grad():
            for inputs, labels in dados:
                inputs, labels = inputs.to(modelo_cliente.device), labels.to(modelo_cliente.device)
                outputs = modelo_cliente(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()

        return total_loss / len(dados)


class Cliente:
    def __init__(self, cliente_id, modelo_global, dados, num_particulas):
        self.cliente_id = cliente_id
        self.modelo = copy.deepcopy(modelo_global)  # Cada cliente tem seu próprio modelo
        self.dados = dados
        self.num_particulas = num_particulas
        self.particulas = []
        self.melhor_particula = None
        self.inicializar_particulas(num_particulas)

    def inicializar_particulas(self, num_particulas):
        """Cria um conjunto de partículas associadas ao cliente."""
        self.particulas = [Particula(i, self.modelo) for i in range(num_particulas)]

    def treinar_com_pso(self, INERCIA, C1, C2, global_best_pesos, criterio):
        """Treina as partículas usando PSO e atualiza a melhor partícula local."""

        for particula in self.particulas:

            particula.atualizar_pso(global_best_pesos, INERCIA, C1, C2)
            erro = particula.avaliar_perda(self.modelo, criterio, self.dados)
            if erro < particula.melhor_erro:
                particula.melhor_erro = erro
                particula.melhor_pesos = copy.deepcopy(particula.pesos)

        self.selecionar_melhor_particula()

    def refinar_com_adam(self, criterio, EPSILON, DELTA, MAX_NORM, SENSITIVITY):
        """Refina os pesos da melhor partícula usando Adam."""
        self.modelo.load_state_dict(self.melhor_particula.melhor_pesos)
        otimizador = optim.Adam(self.modelo.parameters(), lr=0.007, weight_decay=1e-5)

        self.modelo.train()
        for i in range(5):  # 5 épocas de refinamento com Adam
          for inputs, labels in self.dados:
              inputs, labels = inputs.to(self.modelo.device), labels.to(self.modelo.device)
              otimizador.zero_grad()
              outputs = self.modelo(inputs)
              loss = criterio(outputs, labels)
              loss.backward()
              # sigma = SENSITIVITY * torch.sqrt((2.0 * torch.log(torch.tensor(1.0 / DELTA))).clone().detach()) / EPSILON
              # if i==4:
              #   # Percorrer todos os parâmetros do modelo e adicionar ruído aos gradientes
              #   for param in self.modelo.parameters():
              #       if param.grad is not None:
              #           torch.nn.utils.clip_grad_norm_(self.modelo.parameters(), MAX_NORM)
              #           noise = torch.normal(mean=0, std=sigma, size=param.grad.shape, device=param.grad.device)
              #           param.grad += noise  # Adiciona ruído diretamente ao gradiente
              otimizador.step()

        # Atualiza os pesos da melhor partícula com os pesos refinados pelo Adam
        self.melhor_particula.pesos = copy.deepcopy(self.modelo.state_dict())

    def selecionar_melhor_particula(self):
        """Seleciona a melhor partícula do cliente."""
        self.melhor_particula = min(self.particulas, key=lambda p: p.melhor_erro)


def treinar_federado(modelo_global, clientes, criterio, num_rodadas, INERCIA, C1, C2, testloader, EPSILON, DELTA, MAX_NORM, SENSITIVITY):
    """Treina os clientes localmente e sincroniza com o servidor central, validando a acurácia."""

    melhor_peso_global = copy.deepcopy(modelo_global.state_dict())  # Inicializa com o modelo global
    melhor_erro_global = float('inf')

    for rodada in range(num_rodadas):
        resultados_rodada = []

        for cliente in clientes:
            cliente.treinar_com_pso(INERCIA, C1, C2, melhor_peso_global, criterio)  # Treino com PSO
            cliente.refinar_com_adam(criterio, EPSILON, DELTA, MAX_NORM, SENSITIVITY)  # Refinamento com Adam
            erro_cliente = cliente.melhor_particula.melhor_erro  # Obtém o melhor erro do cliente
            resultados_rodada.append((cliente.cliente_id, erro_cliente))

        resultados_sorted = sorted(resultados_rodada, key=lambda x: x[1])
        top_3_results = resultados_sorted[:3]

        melhor_cliente = random.choice(top_3_results)
        melhor_cliente_id = melhor_cliente[0]
        melhor_erro_cliente = melhor_cliente[1]

        melhor_peso_global = copy.deepcopy(clientes[melhor_cliente_id].melhor_particula.pesos)
        melhor_erro_global = melhor_erro_cliente

        modelo_global.load_state_dict(melhor_peso_global)

        test_loss, test_accuracy = avaliar_modelo(modelo_global, criterio, testloader)

        # if (rodada+1) % 10 == 0:
        print(f"Rodada {rodada+1}/{num_rodadas}: Cliente {melhor_cliente_id} enviou os pesos.")
        print(f"Erro Global Atualizado: {melhor_erro_global:.4f}")
        print(f"Teste -> Perda: {test_loss:.4f}, Acurácia: {test_accuracy:.2f}%\n")

    print("Treinamento Federado Finalizado!")

def avaliar_modelo(modelo, criterio, testloader):
    """Avalia o modelo global no conjunto de teste."""
    modelo.eval()  # Modo de avaliação
    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = modelo(inputs)
            loss = criterio(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total_samples += labels.size(0)

    test_loss = total_loss / len(testloader)
    test_accuracy = (correct / total_samples) * 100

    return test_loss, test_accuracy


mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True)
X_train = mnist_train.data.view(-1, 28*28).numpy()  # Flatten (Transforma 28x28 em 784)
y_train = mnist_train.targets.numpy()  # Labels

mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True)
X_test = mnist_test.data.view(-1, 28*28).numpy()  # Flatten (Transforma 28x28 em 784)
y_test = mnist_test.targets.numpy()  # Labels

# Dividir treino e teste manualmente como no Iris

# Normalizar como no Iris
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converter para tensores
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Criar datasets como no Iris
trainset = TensorDataset(X_train, y_train)
testset = TensorDataset(X_test, y_test)

# Função para criar subconjuntos estratificados (balanceados)
def create_stratified_subsets(dataset, num_clients):
    # Extrair os labels do dataset
    targets = dataset.tensors[1].numpy()

    # Dicionário para armazenar os índices de cada classe (0 a 9)
    class_indices = {}
    for digit in range(10):
        indices = np.where(targets == digit)[0]
        np.random.shuffle(indices)  # Embaralha os índices de cada classe
        class_indices[digit] = indices

    # Dicionário para armazenar os índices para cada cliente
    client_indices = {i: [] for i in range(num_clients)}

    # Para cada classe, divide os índices igualmente entre os clientes
    for digit in range(10):
        indices = class_indices[digit]
        splits = np.array_split(indices, num_clients)
        for client_id in range(num_clients):
            client_indices[client_id].extend(splits[client_id].tolist())

    # Cria os subsets utilizando os índices calculados
    subsets = [Subset(dataset, client_indices[i]) for i in range(num_clients)]
    return subsets

train_subsets = create_stratified_subsets(trainset, NUM_CLIENTES)

# Criar DataLoaders para cada subset (cliente)
trainloaders = [DataLoader(train_subsets[i], batch_size=240, shuffle=True) for i in range(NUM_CLIENTES)]

# DataLoader para o conjunto de teste
testloader = DataLoader(testset, batch_size=32, shuffle=False)

# Criando os clientes (assumindo que a classe Cliente, modelo_global e NUM_PARTICULAS já estão definidos)
clientes = [Cliente(i, modelo_global, trainloaders[i], NUM_PARTICULAS) for i in range(NUM_CLIENTES)]

# Executando o treinamento federado
treinar_federado(modelo_global, clientes, criterio, NUM_RODADAS, INERCIA, C1, C2, testloader, EPSILON, DELTA, MAX_NORM, SENSITIVITY)

Rodada 1/10: Cliente 1 enviou os pesos.
Erro Global Atualizado: 0.3405
Teste -> Perda: 0.2938, Acurácia: 96.13%

Rodada 2/10: Cliente 4 enviou os pesos.
Erro Global Atualizado: 0.0887
Teste -> Perda: 0.3326, Acurácia: 95.49%

Rodada 3/10: Cliente 4 enviou os pesos.
Erro Global Atualizado: 0.0444
Teste -> Perda: 0.2961, Acurácia: 95.88%

Rodada 4/10: Cliente 1 enviou os pesos.
Erro Global Atualizado: 0.0558
Teste -> Perda: 0.3360, Acurácia: 95.98%

Rodada 5/10: Cliente 4 enviou os pesos.
Erro Global Atualizado: 0.0260
Teste -> Perda: 0.2989, Acurácia: 96.19%

Rodada 6/10: Cliente 3 enviou os pesos.
Erro Global Atualizado: 0.1240
Teste -> Perda: 0.2655, Acurácia: 96.63%

Rodada 7/10: Cliente 3 enviou os pesos.
Erro Global Atualizado: 0.0326
Teste -> Perda: 0.4163, Acurácia: 95.95%

Rodada 8/10: Cliente 4 enviou os pesos.
Erro Global Atualizado: 0.0260
Teste -> Perda: 0.3791, Acurácia: 96.41%

Rodada 9/10: Cliente 3 enviou os pesos.
Erro Global Atualizado: 0.0303
Teste -> Perda: 0.3293, A